In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy_serving_model_TGI_GCS.ipynb
* [Deploy Gemma 7B with TGI DLC from GCS on Vertex AI](https://github.com/huggingface/Google-Cloud-Containers/blob/main/examples/vertex-ai/notebooks/deploy-gemma-from-gcs-on-vertex-ai/vertex-notebook.ipynb)

In [ ]:
# @title Install Vertex AI SDK and other required packages
%pip install --upgrade --user --quiet google-cloud-aiplatform \
                                      huggingface_hub[hf_transfer] \
                                      transformers

In [ ]:
# @title Define project information
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
BUCKET_URI = "gs://sllm_0103"  # @param {type:"string"}
MODEL_NAME="meta-llama/Llama-3.1-8B-Instruct"  # @param {type:"string"}
ARTIFACT_NAME = "llama3.1_8b_inst"  # @param {type:"string"}

In [ ]:
# @title GCP Authentication

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

In [ ]:
# @title Published TGI Containers
!gcloud container images list --repository="us-docker.pkg.dev/deeplearning-platform-release/gcr.io" | grep "huggingface-text-generation-inference"

In [ ]:
# @title TGI container uri
CONTAINER_URI = "us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu124.2-4.ubuntu2204.py311" # @param {type:"string"}

In [ ]:
# @title Enable apis
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable compute.googleapis.com
!gcloud services enable container.googleapis.com
!gcloud services enable containerregistry.googleapis.com
!gcloud services enable containerfilesystem.googleapis.com

In [ ]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

In [ ]:
# @title Authenticate your Hugging Face account
from huggingface_hub import interpreter_login
interpreter_login()

After `huggingface_hub` installation and login are completed, you can run the following bash script to download the model locally within a temporary directory, and then upload those to the GCS Bucket.

### Download a model in Hugging face into GCS

In [ ]:
# @title Download configuration
LOCAL_DIR="model/llama3.1-8b-it"
! mkdir -p {LOCAL_DIR}

In [ ]:
# @title Download model from Hugging face

! huggingface-cli download meta-llama/Llama-3.1-8B-Instruct --exclude "*.bin" "*.pth" "*.gguf" ".gitattributes" --local-dir {LOCAL_DIR}

In [ ]:
# @title Upload model files into GCS
! gsutil -o GSUtil:parallel_composite_upload_threshold=150M -m cp -e -r {LOCAL_DIR}/* {BUCKET_URI}/{ARTIFACT_NAME}

## Register model on Vertex AI

In [ ]:
# @title Initalize vertex ai
import os
from google.cloud import aiplatform

aiplatform.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=BUCKET_URI,
)

In [ ]:
# @title Upload a model to registry on Vertex AI
model = aiplatform.Model.upload(
    display_name="llama3.1_8b_inst",
    artifact_uri=f"{BUCKET_URI}/{ARTIFACT_NAME}",
    serving_container_image_uri=CONTAINER_URI,
    serving_container_environment_variables={
        "NUM_SHARD": "1",
        "MAX_INPUT_TOKENS": "512",
        "MAX_TOTAL_TOKENS": "1024",
        "MAX_BATCH_PREFILL_TOKENS": "1512",
    },
    serving_container_ports=[8080],
)
model.wait()

## Deploy model on Vertex AI

In [ ]:
# @title Create endpoint
endpoint_name = "llama3.1_8b_inst-endpoint"
endpoint = aiplatform.Endpoint.create(display_name=endpoint_name)

In [ ]:
# @title Deploy a model
deployed_model = model.deploy(
    endpoint=endpoint,
    machine_type="g2-standard-4",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
)

## Online predictions on Vertex AI

In [ ]:
# @title Get tokenizer
import os
from huggingface_hub import get_token
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=get_token(),
)

In [ ]:
# @title Predict within the same session
messages = [
    {"role": "user", "content": "What's Deep Learning?"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
# <bos><start_of_turn>user\nWhat's Deep Learning?<end_of_turn>\n<start_of_turn>model\n

output = deployed_model.predict(
    instances=[
        {
            "inputs": inputs,
            "parameters": {
                "max_new_tokens": 256, "do_sample": True,
                "top_p": 0.95, "temperature": 1.0,
            },
        },
    ]
)
print(output.predictions[0])

In [ ]:
# @title Predict within a different session
import os
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

endpoint_display_name = "llama3.1_8b_inst-endpoint"  # TODO: change to your endpoint display name

# Iterates over all the Vertex AI Endpoints within the current project and keeps the first match (if any), otherwise set to None
ENDPOINT_ID = next(
    (endpoint.name for endpoint in aiplatform.Endpoint.list()
     if endpoint.display_name == endpoint_display_name),
    None
)

print(ENDPOINT_ID)

assert ENDPOINT_ID, (
    "`ENDPOINT_ID` is not set, please make sure that the `endpoint_display_name` is correct at "\
    f"https://console.cloud.google.com/vertex-ai/online-prediction/endpoints?project={PROJECT_ID}"
)

endpoint = aiplatform.Endpoint(f"projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}")
print(endpoint)

output = endpoint.predict(
    instances=[
        {
            "inputs": "<bos><start_of_turn>user\nWhat's Deep Learning?<end_of_turn>\n<start_of_turn>model\n",
            "parameters": {
                "max_new_tokens": 128,
                "do_sample": True,
                "top_p": 0.95,
                "temperature": 0.7,
            },
        },
    ],
)
print(output.predictions[0])

In [ ]:
# @title Resource clean up
deployed_model.undeploy_all()
deployed_model.delete()
model.delete()

In [ ]:
!gcloud storage rm -r $BUCKET_URI